In [19]:
import pyarrow.parquet as pq
import pyarrow as pa
import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# Define chunk size (number of rows to read and modify at a time)
chunk_size = 64_000  # Adjust this according to your needs

parquet_file = pq.ParquetFile('pairs_df_with_rows.parquet')

total_rows = parquet_file.metadata.num_rows

print(total_rows)

output_writer = None

rows = []

count = 0

for batch in tqdm.tqdm(parquet_file.iter_batches(chunk_size), total=int(total_rows / chunk_size)):
    # add similarity double column to batch without making into pandas
    # print(batch)
    # print(type(batch))

    similarity_column = pa.array([-1.0] * len(batch))

    batch_with_results = pa.RecordBatch.from_arrays(
        batch.columns + [similarity_column],  # add new column to batch
        schema=batch.schema.append(pa.field("similarity", 'double'))
    )

    # Append the modified chunk_table to the output Parquet file
    if not output_writer:
        # Create or open the output Parquet file
        output_writer = pq.ParquetWriter(
            'pairs_with_empty_similarity.parquet',
            batch_with_results.schema,
        )
    output_writer.write_table(
        pa.Table.from_batches([batch_with_results])
    )
    
    # count += 1
    # if count > 3:
    #     print(len(rows))
    #     break
        # FIXME: for dev

# Close the output Parquet writer
output_writer.close()

66932677


1046it [00:29, 34.95it/s]                          


In [17]:
import pandas as pd
df = pd.read_parquet('pairs_with_empty_similarity.parquet')
df.head()


,census_id,deepdao_organization_id,deepdao_name,census_name,row_id,similarity
0,1fe442f5-891f-5a6f-b3a1-1b846a7a8b70,da9956dc-8a87-40c0-a066-b8991d67e574,PancakeSwap,Stargate DAO,1fe442f5-891f-5a6f-b3a1-1b846a7a8b70_da9956dc-...,-1.0
1,1fe442f5-891f-5a6f-b3a1-1b846a7a8b70,a356feb3-546c-457b-a1c5-cde5bb39d5df,Nonsensical Snapshots (8),Stargate DAO,1fe442f5-891f-5a6f-b3a1-1b846a7a8b70_a356feb3-...,-1.0
2,1fe442f5-891f-5a6f-b3a1-1b846a7a8b70,60c9b31c-4495-4028-aeac-eb7bb117fece,Decentraland,Stargate DAO,1fe442f5-891f-5a6f-b3a1-1b846a7a8b70_60c9b31c-...,-1.0
3,1fe442f5-891f-5a6f-b3a1-1b846a7a8b70,ee239a92-7cb8-4ba6-bdcc-77a3a8341613,Nonsensical Snpashots: lumao 狂奔的蜗牛 ecosystem,Stargate DAO,1fe442f5-891f-5a6f-b3a1-1b846a7a8b70_ee239a92-...,-1.0
4,1fe442f5-891f-5a6f-b3a1-1b846a7a8b70,ad814279-3f12-428d-820a-be25abb51273,MetaCartel Ventures,Stargate DAO,1fe442f5-891f-5a6f-b3a1-1b846a7a8b70_ad814279-...,-1.0
